In [7]:
# 엔트로피 방법으로 4가지 요소 가중치 계산
import pandas as pd
import numpy as np

# 1. 데이터 입력
data = {
    '군구': ['강화군','계양구','남동구','동구','미추홀구','부평구','서구','연수구','옹진군','중구'],
    '노인여가복지시설_부족지수': [0.14893617,0.463829787,0.357446809,1,0.438297872,0.365957447,0,0.468085106,0.855319149,0.761702128],
    '재가복지시설_부족시수': [0.892,0.403,0,1,0.144,0.489,0.05,0.518,0.964,0.597],
    '주거복지시설_부족지수': [0,0.75,0.875,1,1,0.625,0.375,0.875,0.875,1],
    '의료복지시설_부족지수': [0.781,1.057,0.848,1,0.61,0.743,0,0.486,0.457,0.276],
    '의료복지시설_입소율_부족지수': [0.38,0.18,0.19,0,0.3,0.15,0.28,0.26,1,0.18],
    '주거복지시설_입소율_부족지수': [0.55,0.09,0.29,1,1,0.26,0,0.76,1,1],
    '범죄지수': [0,0.336,0.691,0.375,0.764,1,0.773,0.472,0.695,0.375],
    '독거노인비율_정규화': [0.252669039,0.57,0.69,0.930604982,0.5,0.64,0.04,0,1,0.79]
}

# 2. 데이터프레임 생성
df = pd.DataFrame(data)

# 3. 분석 대상 변수 (군구 제외)
features = df.columns.drop('군구')
X = df[features].values

# 4. 엔트로피 가중치 계산
P = X / X.sum(axis=0)
epsilon = 1e-12
E = - (P * np.log(P + epsilon)).sum(axis=0) / np.log(len(df))
d = 1 - E
weights = d / d.sum()

# 5. 가중치 출력
print("각 변수별 엔트로피 가중치:")
for col, w in zip(features, weights):
    print(f"{col}: {w:.3f}")

# 6. 위험지수 계산
df['위험지수'] = X @ weights

# 7. 결과 출력
print("\n군구별 위험지수 (높은 순):")
print(df[['군구', '위험지수']].sort_values(by='위험지수', ascending=False).reset_index(drop=True))


각 변수별 엔트로피 가중치:
노인여가복지시설_부족지수: 0.119
재가복지시설_부족시수: 0.162
주거복지시설_부족지수: 0.075
의료복지시설_부족지수: 0.093
의료복지시설_입소율_부족지수: 0.174
주거복지시설_입소율_부족지수: 0.148
범죄지수: 0.091
독거노인비율_정규화: 0.138

군구별 위험지수 (높은 순):
     군구      위험지수
0   옹진군  0.889421
1    동구  0.759112
2    중구  0.610213
3  미추홀구  0.545492
4   부평구  0.482636
5   연수구  0.450711
6   계양구  0.428608
7   남동구  0.420817
8   강화군  0.417033
9    서구  0.160965


In [15]:
import pandas as pd
import folium
import json

# 고독사 위험지수 데이터 구성
risk_data = {
    "군구": ["강화군", "계양구", "남동구", "동구", "미추홀구", "부평구", "서구", "연수구", "옹진군", "중구"],
    #"고독사위험지수": [0.158, 0.459, 0.611, 0.576, 0.641, 0.604, 0.367, 0.402, 0.892, 0.586],
    "엔트로피고독사위험지수": [0.417,0.429,0.421,0.759,0.545,0.483,0.161,0.451,0.889,0.610]
}
risk_df = pd.DataFrame(risk_data)

# GeoJSON 파일 불러오기 (인천광역시 군구 경계 파일)
with open("인천광역시_군구_경계.geojson", encoding="utf-8") as f:
    geo_data = json.load(f)

# 지도 생성
m = folium.Map(location=[37.45, 126.70], zoom_start=10)

# Choropleth 시각화 : 고독사 위험지수 기준 색상 맵핑
choropleth = folium.Choropleth(
    geo_data=geo_data,
    name="choropleth",
    data=risk_df,
    columns=["군구", "엔트로피고독사위험지수"],
    key_on="feature.properties.sggnm",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="엔트로피 고독사 위험지수",
    highlight=True,
    # threshold_scale=[0.15, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.75, 0.8, 0.9, 0.95]
).add_to(m)

# 지도에 툴팁 추가
folium.GeoJsonTooltip(fields=["sggnm"], aliases=["군구:"]).add_to(choropleth.geojson)

# 저장
m.save("엔트로피_고독사_위험지수_시각화.html")